In [ ]:
import tensorflow as tf
import numpy as np
import gym
import collections
import random
import os

In [ ]:
ffmpeg_path = 'C:\\Users\\dhzns\\OneDrive\\code\\ipython\\OpenAI_gym\\ffmpeg\\bin'
os.environ['PATH'] = ffmpeg_path + ';' + os.environ['PATH']

In [3]:
#game_type
game = 'CartPole-v1'

#File Path
filepath_ckpt  = './ckpt/CartPole_yt.ckpt' #weight saver check point file path
filepath_monitor = './tmp/cartpole-v1-experiment1' # monitor file path

#hyper params
batch_size = 20
learning_rate = 0.01
input_size = 4
output_size = 2
h_size = 64
discount_rate = 0.80

max_episode = 500
REPLAY_MEMORY = 2000

#api key
api_key = 'sk_R6pSeGWSRfSWyCxZP08FiQ'

In [4]:
env = gym.make(game)
replay_buffer = collections.deque()

[2017-08-30 21:39:06,989] Making new env: CartPole-v1


In [5]:
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network(h_size = h_size, l_rate = learning_rate)
        
    def _build_network(self, h_size=10, l_rate=0.05):
        with tf.name_scope(self.net_name) as scope:
            self._X = tf.placeholder(tf.float32, [None, input_size], name='Observation_placeholder')
            self._Y = tf.placeholder(tf.float32, [None, output_size], name='Q_placeholder')

            W1 = tf.Variable(tf.truncated_normal([self.input_size, h_size], dtype=tf.float32), trainable=True, name='weight')
            l1p = tf.matmul(self._X,W1)
            l1 = tf.nn.tanh(l1p)
        
#             tf.contrib.layers.batch_norm(l1)
            W2 = tf.Variable(tf.truncated_normal([h_size, output_size], dtype=tf.float32), trainable=True, name='weight')
            l2p = tf.matmul(l1, W2)
#             l2 = tf.nn.relu(l2p)
      
#             tf.contrib.layers.batch_norm(l2)
#             W3 = tf.Variable(tf.truncated_normal([h_size, h_size], dtype=tf.float32), trainable=True, name='weight')
#             l3p = tf.matmul(l2, W3)
#             l3 = tf.nn.relu(l3p)
        
#             tf.contrib.layers.batch_norm(l3)
#             W4 = tf.Variable(tf.truncated_normal([h_size, self.output_size], dtype=tf.float32), trainable=True, name='weight')
#             l4p = tf.matmul(l3, W4)
#             l4 = tf.nn.relu(l4p)
            
            self.logits = l2p
#             self.prob = tf.nn.tanh(self.logits)

            self._loss = tf.reduce_sum(tf.square(self._Y - self.logits))
            self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
            
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self.logits, feed_dict={self._X: x})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

In [6]:
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        if done:
            Q[0, action]= reward
        else:
            Q[0, action]= reward + discount_rate * np.max(targetDQN.predict(next_state))
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        
    return mainDQN.update(x_stack, y_stack)

In [7]:
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
        
    return op_holder

In [8]:
sess = tf.Session()
mainDQN = DQN(sess, input_size, output_size, name="main")
targetDQN = DQN(sess, input_size, output_size, name="target")

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

# saver.restore(sess, filepath_ckpt)

In [9]:
copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name = "main")
sess.run(copy_ops)
p= 0.2
for i_episode in range(max_episode):
    if p > 0:
        p -=0.01
    s = env.reset()
    done = False
    t=0
    while not done:
#         env.render()
        t += 1
        e = random.random()
#         print(e)
        if e < p:
            a = env.action_space.sample()
        else:
#             print("select")
            a = np.argmax(mainDQN.predict(s))
#         print(action)
        
        #get new state    
        s1, rwd, done, _ = env.step(a)
        
        if done:
            if t < 15:
                rwd -= -250
            if t < 50:
                rwd -= -200
            if t < 100:
                rwd -= -150
            if t < 250:
                rwd -= -100
            if t < 400:
                rwd -= -50
        replay_buffer.append((s, a, rwd, s1, done))
        if len(replay_buffer) > REPLAY_MEMORY:
            replay_buffer.popleft()            
        s = s1
        
    if i_episode % 2 == 0: 
        print("Episode finished after {} timesteps".format(t))

        for _ in range(50):
            mini_batch = random.sample(replay_buffer, batch_size)
            loss, _ = replay_train(mainDQN, targetDQN, mini_batch)
        print("Loss : ", loss)
        sess.run(copy_ops)

Episode finished after 91 timesteps
Loss :  229.05
Episode finished after 318 timesteps
Loss :  114.218
Episode finished after 208 timesteps
Loss :  83.989
Episode finished after 239 timesteps
Loss :  306.212
Episode finished after 500 timesteps
Loss :  78.9167
Episode finished after 500 timesteps
Loss :  82.421
Episode finished after 500 timesteps
Loss :  19.8199
Episode finished after 500 timesteps
Loss :  20.14
Episode finished after 500 timesteps
Loss :  37.6695
Episode finished after 500 timesteps
Loss :  61.1692
Episode finished after 500 timesteps
Loss :  27.8244
Episode finished after 500 timesteps
Loss :  33.6481
Episode finished after 500 timesteps
Loss :  20.4575
Episode finished after 500 timesteps
Loss :  23.6765
Episode finished after 500 timesteps
Loss :  29.7396
Episode finished after 500 timesteps
Loss :  18.1918
Episode finished after 500 timesteps
Loss :  8.28021
Episode finished after 383 timesteps
Loss :  20.8711
Episode finished after 500 timesteps
Loss :  19.2732

Loss :  23.9478
Episode finished after 500 timesteps
Loss :  13.6165
Episode finished after 500 timesteps
Loss :  50.2853
Episode finished after 281 timesteps
Loss :  34.1311
Episode finished after 329 timesteps
Loss :  22.3796
Episode finished after 290 timesteps
Loss :  28.1192
Episode finished after 312 timesteps
Loss :  74.7299
Episode finished after 500 timesteps
Loss :  68.4202
Episode finished after 292 timesteps
Loss :  161.47
Episode finished after 249 timesteps
Loss :  71.8215
Episode finished after 375 timesteps
Loss :  32.4918
Episode finished after 286 timesteps
Loss :  102.624
Episode finished after 473 timesteps
Loss :  16.5215
Episode finished after 500 timesteps
Loss :  32.4906
Episode finished after 500 timesteps
Loss :  36.373
Episode finished after 500 timesteps
Loss :  64.064
Episode finished after 394 timesteps
Loss :  48.9454
Episode finished after 500 timesteps
Loss :  50.7376
Episode finished after 500 timesteps
Loss :  21.2803
Episode finished after 500 timest

In [10]:
saver.save(sess, filepath_ckpt)

'./ckpt/CartPole_yt.ckpt'

In [11]:
env.close()

In [12]:
gym.upload(filepath_monitor, api_key=api_key)

[2017-08-30 21:41:56,955] [CartPole-v1] Uploading 1001 episodes of training data
[2017-08-30 21:41:58,563] [CartPole-v1] Uploading videos of 11 training episodes (200709 bytes)
[2017-08-30 21:41:59,447] [CartPole-v1] Creating evaluation object from ./tmp/cartpole-v1-experiment1 with learning curve and training video
[2017-08-30 21:41:59,751] 
****************************************************
You successfully uploaded your evaluation on CartPole-v1 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_l1Mt1XRoTJa52LzMeT2wYQ

****************************************************


In [13]:
# for the test
env = gym.make(game)
for i_episode in range(20):
    s = env.reset()
    for t in range(501):
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, rwd, done, _ = env.step(a)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

[2017-08-30 21:41:59,781] Making new env: CartPole-v1


Episode finished after 394 timesteps
Episode finished after 488 timesteps
Episode finished after 396 timesteps
Episode finished after 400 timesteps
Episode finished after 284 timesteps
Episode finished after 283 timesteps
Episode finished after 258 timesteps
Episode finished after 455 timesteps
Episode finished after 500 timesteps
Episode finished after 387 timesteps
Episode finished after 468 timesteps
Episode finished after 500 timesteps
Episode finished after 365 timesteps
Episode finished after 444 timesteps
Episode finished after 500 timesteps
Episode finished after 381 timesteps
Episode finished after 322 timesteps
Episode finished after 439 timesteps
Episode finished after 450 timesteps
Episode finished after 500 timesteps


In [14]:
env.close()
sess.close()